In [14]:
from typing import List, Tuple

def format_chat_history(chat_history: List[Tuple]) -> str:
    buffer = ""
    for dialogue_turn in chat_history:
        human = "Human: " + dialogue_turn[0]
        ai = "Assistant: " + dialogue_turn[1]
        buffer += "\n".join([human, ai]) + "\n"
    return buffer

In [15]:

data = [('1', '2', 'a'), ('3', '4'), ('5', '6')]
print(format_chat_history(data))


Human: 1
Assistant: 2
Human: 3
Assistant: 4
Human: 5
Assistant: 6

